# Data Cleaning
---
Take the raw data from `/data/hamSpam.csv` and `/data/phish.csv` and conform the ham/spam dataset to the structure of the phishing dataset and integrate together. Then clean the dataset by removing NA values, standardize the textual features by lowercasing and removing stopwords, and remove any embedded HTML elements from the text.

## 1. Imports & Setup

In [ ]:
import pandas as pd

import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

from utils import clean_text

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prokope/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Download nltk datasets.

In [49]:
# NLTK downloads
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /Users/prokope/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/prokope/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/prokope/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## 2. Load Data from CSV

In [50]:
hamSpam_df = pd.read_csv("./data/1_hamSpam.csv")
phish_df = pd.read_csv("./data/1_phish.csv")

## 3. Process Ham/Spam Data

In [51]:
# Rename known columns (no error if source names are missing)
hamSpam_df.rename(
    columns={"Spam/Ham": "Email Type", "Message": "Email Text"},
    inplace=True
)

# Drop optional columns only if they exist to avoid KeyError
cols_to_drop = [c for c in ["Date", "Subject"] if c in hamSpam_df.columns]

if cols_to_drop:
	hamSpam_df.drop(columns=cols_to_drop, inplace=True)

# Filter out emails that are not ham/spam
hamSpam_df = hamSpam_df[hamSpam_df['Email Type'].isin(['ham', 'spam'])]

# Export as processed dataset
hamSpam_df.to_feather(
    "data/2_hamSpam_processed.feather"
)

## 4. Process Phishing Data

Observe email types.

In [52]:
phish_df['Email Type'].apply(repr).unique()

array(["'Safe Email'", "'Phishing Email'"], dtype=object)

Standardize email types.

In [53]:
phish_df['Email Type'] = phish_df['Email Type'].replace(
    {'Safe Email': 'ham', 'Phishing Email': 'phish'}
)
phish_df['Email Type'].apply(repr).unique()

array(["'ham'", "'phish'"], dtype=object)

## 5. Combine Datasets

In [54]:
email_df = pd.concat([hamSpam_df, phish_df], ignore_index=True)
email_df['Email Type'].apply(repr).unique()

array(["'ham'", "'spam'", "'phish'"], dtype=object)

Show total count of emails.

In [55]:
email_df.count()

Email Text    52298
Email Type    52366
dtype: int64

## 6. Drop NA and "empty" Rows

In [56]:
print(email_df.isna().sum())

email_df.dropna(inplace=True)

empty_rows = email_df[email_df["Email Text"] == "empty"]
email_df.drop(empty_rows.index, inplace=True)

print(email_df.isna().sum())

Email Text    68
Email Type     0
dtype: int64
Email Text    0
Email Type    0
dtype: int64


## 7. Bulk Clean Dataset & Export to CSV


Clean data by removing stop words, punctuation, special characters, and tokenize/lemmatize.

In [57]:
cleaned_email_df = email_df.copy()
cleaned_email_df["Cleaned Email Text"] = email_df["Email Text"].apply(clean_text)
cleaned_email_df

,Email Text,Email Type,Cleaned Email Text
1,"gary , production from the high island larger ...",ham,gary production high island larger block comme...
2,- calpine daily gas nomination 1 . doc,ham,calpine daily gas nomination doc
3,fyi - see note below - already done .\nstella\...,ham,fyi see note already done stella forwarded ste...
4,fyi .\n- - - - - - - - - - - - - - - - - - - -...,ham,fyi forwarded lauri allen hou ect pm kimberly ...
5,"jackie ,\nsince the inlet to 3 river plant is ...",ham,jackie since inlet river plant shut last day f...
...,...,...,...
52360,\nRick Moen a Ã©crit:> > I'm confused. I thou...,ham,rick moen im confused thought gpled money paid...
52361,date a lonely housewife always wanted to date ...,phish,date lonely housewife always wanted date lonel...
52362,request submitted : access request for anita ....,ham,request submitted access request anita dupont ...
52363,"re : important - prc mtg hi dorn & john , as y...",ham,important prc mtg hi dorn john discovered rece...


Export to feather.

In [58]:
cleaned_email_df.to_feather(
    "./data/2_clean_email_dataset.feather"
)